In [1]:
import numpy as np
import pandas as pd
import emoji
import datetime

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split  # Import for splitting the data
from gensim.models import KeyedVectors

2024-02-28 01:30:33.161710: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-28 01:30:33.255545: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 01:30:33.255600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 01:30:33.258039: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 01:30:33.272618: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
data = pd.read_csv("data/emoji_data/emoji_data.csv", header=None)
data.head()

Emoji Dictionary

In [10]:

emoji_dictionary = {
    0: ":red_heart:",  # Love #
    1: ":face_with_tears_of_joy:",  # Laughter
    2: ":grinning_face_with_big_eyes:", # Happiness #
    3: ":loudly_crying_face:",  # Sadness #
    4: ":smiling_face_with_heart-eyes:",  # Adoration
    5: ":fire:",  # Excitement
    6: ":thumbs_up:",  # Approval
    7: ":folded_hands:",  # Gratitude
    8: ":angry_face:",  # Anger
    9: ":thinking_face:",  # Contemplation
}


In [ ]:
emoji_dictionary = {
    0: ":red_heart:",  # Love #
    1: ":face_with_tears_of_joy:",  # Laughter
    2: ":grinning_face_with_big_eyes:", # Happiness #
    3: ":loudly_crying_face:",  # Sadness #
    4: ":smiling_face_with_heart-eyes:",  # Adoration
    5: ":fire:",  # Excitement
    6: ":thumbs_up:",  # Approval
    7: ":folded_hands:",  # Gratitude
    8: ":angry_face:",  # Anger
    9: ":thinking_face:",  # Contemplation
}


Function to get labels from CLDR names

In [11]:
def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

Reading data

In [12]:
data = pd.read_csv('data/emoji_data/emoji_data.csv', header=None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,Good joke,1
3,what is your favorite baseball game,9
4,I cooked meat,2


In [13]:

def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

In [14]:
X = data[0].values
Y = data[1].values


Embeddings

With glove dataset

In [15]:
# with open('data/glove_dataset/glove.6B.100d.txt','r', encoding='utf8') as file:
#     content = file.readlines()

In [16]:
# embeddings = {}

# for line in content:
#     line = line.split()
#     embeddings[line[0]] = np.array(line[1:], dtype=float)

With crawl dataset

In [17]:
fasttext_model_path = "data/fast_text/crawl_dataset/crawl-300d-2M-subword.vec"
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path)


In [18]:
# Convert FastText embeddings to a dictionary
embeddings = {}
for word in fasttext_model.index_to_key:
    embeddings[word] = fasttext_model.get_vector(word)


Convert input text into tokens

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word_to_index = tokenizer.word_index


In [20]:
Xtokens = tokenizer.texts_to_sequences(X)


In [21]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    
    return maxlen
maxlen = get_maxlen(Xtokens)

maxlen

11

In [22]:
Xtrain = pad_sequences(Xtokens, maxlen=maxlen, padding='post', truncating='post')


In [23]:
Ytrain = to_categorical(Y)

Split data into train and test set

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(Xtrain, Ytrain, test_size=0.2, random_state=42)

Model

In [25]:
embed_size = 300

In [26]:
embedding_matrix = np.zeros((len(word_to_index) + 1, embed_size))

for word, i in word_to_index.items():
    if word in embeddings:
        embed_vector = embeddings[word]
        embedding_matrix[i] = embed_vector
    else:
        # Handle out-of-vocabulary words or phrases by aggregating subword embeddings
        phrase_embed_sum = None
        for subword in word.split():
            if subword in embeddings:
                if phrase_embed_sum is None:
                    phrase_embed_sum = embeddings[subword]
                else:
                    phrase_embed_sum += embeddings[subword]
        if phrase_embed_sum is not None:
            # Take the average of subword embeddings
            embedding_matrix[i] = phrase_embed_sum / len(word.split())

Code to generate file name for saving model

In [27]:
def returnfilename(prefix=""):
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"{formatted_datetime}_epochs_{epochs}_layers_{layer1}_{layer2}_{layer3}_{layer4}"
    return filename


In general more layers >>

In [31]:
epochs = 50
#units
layer1 = 256
layer2 = 16
layer3 = 4
layer4 = 2

num_classes = len(emoji_dictionary)

In [33]:
model = Sequential([
    Embedding(input_dim=len(word_to_index) + 1,
              output_dim=embed_size,
              input_length=maxlen,
              weights=[embedding_matrix],
              trainable=False),
    LSTM(units=layer1, return_sequences=True),
    LSTM(units=layer2, return_sequences=True),
    LSTM(units=layer3, return_sequences=True),
    LSTM(units=layer4),
    Dense(num_classes, activation='softmax')  # Set output dimensionality to 20
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.fit(Xtrain, Ytrain, epochs=epochs)

Epoch 1/50


2024-02-28 00:41:56.402816: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-28 00:41:56.849195: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8fe00b30d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-28 00:41:56.849251: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-02-28 00:41:56.870531: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709077317.115656   23123 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 [==============================] - 9s 34ms/step - loss: 2.2994 - accuracy: 0.1221
Epoch 2/50
13/13 [==============================] - 0s 20ms/step - loss: 2.2819 - accuracy: 0.1481
Epoch 3/50
13/13 [==============================] - 0s 16ms/step - loss: 2.2602 - accuracy: 0.1481
Epoch 4/50
13/13 [==============================] - 0s 16ms/step - loss: 2.2428 - accuracy: 0.1688
Epoch 5/50
13/13 [==============================] - 0s 15ms/step - loss: 2.2265 - accuracy: 0.1740
Epoch 6/50
13/13 [==============================] - 0s 15ms/step - loss: 2.2241 - accuracy: 0.1662
Epoch 7/50
13/13 [==============================] - 0s 16ms/step - loss: 2.2194 - accuracy: 0.1714
Epoch 8/50
13/13 [==============================] - 0s 15ms/step - loss: 2.2459 - accuracy: 0.1273
Epoch 9/50
13/13 [==============================] - 0s 13ms/step - loss: 2.2246 - accuracy: 0.1610
Epoch 10/50
13/13 [==============================] - 0s 16ms/step - loss: 2.2056 - accuracy: 0.1662
Epoch 11/50
13/13 [=

In [35]:
filename = returnfilename()

model_json = model.to_json()
with open(f"models/json/{filename}.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(f"models/weights/{filename}.h5")

In [36]:
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

3/3 [==============================] - 2s 10ms/step - loss: 2.1009 - accuracy: 0.2078
Test Loss: 2.100893974304199
Test Accuracy: 0.20779220759868622


In [37]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted and actual labels from one-hot encoded format to integer labels
y_pred_labels = np.argmax(y_pred, axis=1)
Y_test_labels = np.argmax(Y_test, axis=1)

# Compare predicted labels with actual labels
correct_predictions = np.sum(y_pred_labels == Y_test_labels)
total_predictions = len(Y_test_labels)
accuracy = correct_predictions / total_predictions

# Print accuracy
print("Accuracy:", accuracy)

# Display predicted labels and actual labels
for i in range(len(y_pred_labels)):
    if y_pred_labels[i]!=Y_test_labels[i]:
        print("Predicted:", label_to_emoji(y_pred_labels[i]), "Actual:", label_to_emoji(Y_test_labels[i]))


3/3 [==============================] - 2s 6ms/step
Accuracy: 0.2077922077922078
Predicted: 😃 Actual: ❤️
Predicted: 🤔 Actual: 😠
Predicted: 😃 Actual: 👍
Predicted: 😃 Actual: 😍
Predicted: 😃 Actual: 🤔
Predicted: 🔥 Actual: 🙏
Predicted: 😃 Actual: 😠
Predicted: 😃 Actual: 🙏
Predicted: 😃 Actual: 😠
Predicted: 😃 Actual: 🔥
Predicted: 😃 Actual: ❤️
Predicted: 🔥 Actual: 😭
Predicted: 😃 Actual: 🤔
Predicted: 🤔 Actual: 😠
Predicted: 😃 Actual: 👍
Predicted: 😃 Actual: 👍
Predicted: 😃 Actual: 😠
Predicted: 🤔 Actual: 😠
Predicted: 🔥 Actual: 😃
Predicted: 🔥 Actual: 😭
Predicted: 🤔 Actual: 😂
Predicted: 😍 Actual: 😃
Predicted: 🙏 Actual: 😭
Predicted: 🔥 Actual: 👍
Predicted: 😍 Actual: 😃
Predicted: 🔥 Actual: 👍
Predicted: 😃 Actual: ❤️
Predicted: 😃 Actual: 🔥
Predicted: 😃 Actual: 🔥
Predicted: 😃 Actual: ❤️
Predicted: 😍 Actual: 👍
Predicted: 🤔 Actual: 😠
Predicted: 🔥 Actual: 😃
Predicted: 😃 Actual: 🔥
Predicted: 😃 Actual: 😭
Predicted: 😃 Actual: ❤️
Predicted: 😍 Actual: 🔥
Predicted: 😃 Actual: 😍
Predicted: 😃 Actual: 👍
Predicted: 😃 Actua

In [38]:
# Make predictions
test = ["I am trying", "I want to cry", "This is just sad"]
test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen=maxlen, padding='post', truncating='post')
y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis=1)

1/1 [==============================] - 0s 34ms/step


In [39]:
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I am trying 😃
I want to cry 🤔
This is just sad 🤔
